In [ ]:
#!pip install nibabel
#!pip install nilearn
#!pip install opencv-python
#!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
#!pip install segmentation-models-pytorch
#!pip install imageio
#!pip install matplotlib
#!pip install torchmetrics
#!pip install natsort
!pip install albumentations

In [1]:
import nibabel as nib
import cv2
import numpy as np
import os
import imageio.v2 as imageio
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import segmentation_models_pytorch as smp
import torch.optim as optim
import torch.nn as nn
from sklearn.model_selection import train_test_split
import shutil
import matplotlib.pyplot as plt
from torchmetrics.classification import Dice
from natsort import natsorted
import albumentations as A
from skimage.transform import resize

/opt/anaconda/miniconda3/envs/TFG/lib/python3.12/site-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.3 (you have 1.4.22). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
def porcentaje(numero, porcentaje):
    num = numero * (porcentaje/100)
    return int(num)

In [3]:
def obtener_ficheros(carpeta):
    lista_ficheros = []
    for archivo in os.listdir(carpeta):
        ruta = os.path.join(carpeta, archivo)
        if archivo.lower().endswith('.png'):
            lista_ficheros.append(ruta)
    return natsorted(lista_ficheros)

In [4]:
class Custom_Dataset(Dataset):
    def __init__(self, imagenes, mascaras, transform=None, argument=False):
        self.imagenes = imagenes
        self.mascaras = mascaras
        self.transform = transform
        self.argument = argument
        
    def __len__(self):
        return len(self.imagenes)
        
    def __getitem__(self, idx):
        img = imageio.imread(self.imagenes[idx])
        mask = imageio.imread(self.mascaras[idx])

        target_height = 224
        target_width = 224

        img = cv2.resize(img, (target_width, target_height), interpolation = cv2.INTER_AREA)
        mask = cv2.resize(mask, (target_width, target_height), interpolation = cv2.INTER_NEAREST)
        
        img = img/255.0
        mask = mask/255.0
        
        if self.argument:
            transform = A.Compose([
                A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=15, p=0.5),
                A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
                A.CLAHE(clip_limit=2.0, p=0.5)
            ])

            img_for_aug = (img * 255).astype(np.uint8)
            mask_for_aug = (mask * 255).astype(np.uint8)

            augmented = transform(image=img_for_aug, mask=mask_for_aug)

            img = augmented['image'].astype(np.float32) / 255.0
            mask = augmented['mask'].astype(np.float32) / 255.0

        img = np.expand_dims(img, axis=-1)
        mask = np.expand_dims(mask, axis=-1)
        
        img_tensor = torch.from_numpy(img)
        img_tensor = img_tensor.permute(2, 0, 1).float()
        mask_tensor = torch.from_numpy(mask)
        mask_tensor = mask_tensor.permute(2, 0, 1).float()

        return img_tensor, mask_tensor

In [5]:
carpeta = 'Resultados2/Cerebro/Axial/'
lista_ficheros_cerebro_axial = obtener_ficheros(carpeta)
print(len(lista_ficheros_cerebro_axial))

7920


In [6]:
carpeta = 'Resultados2/Mascara/Axial/'
lista_ficheros_mascara_axial = obtener_ficheros(carpeta)
print(len(lista_ficheros_mascara_axial))

7920


In [7]:
carpeta = 'Resultados2/Cerebro/Axial/Val/'
lista_ficheros_cerebro_axial_val = obtener_ficheros(carpeta)
print(len(lista_ficheros_cerebro_axial_val))

1100


In [8]:
carpeta = 'Resultados2/Mascara/Axial/Val/'
lista_ficheros_mascara_axial_val = obtener_ficheros(carpeta)
print(len(lista_ficheros_mascara_axial_val))

1100


In [9]:
train_dataset = Custom_Dataset(lista_ficheros_cerebro_axial,lista_ficheros_mascara_axial, argument=True)
dataloader_train = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [10]:
val_dataset = Custom_Dataset(lista_ficheros_cerebro_axial_val,lista_ficheros_mascara_axial_val)
dataloader_val = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [11]:
model = smp.Unet(
    encoder_name="resnet34",
    encoder_weights="imagenet",
    in_channels=1,
    classes=1
)

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [13]:
model = model.to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
dice_metric = Dice(average='micro').to(device)

In [5]:
class EarlyStopping():
    def __init__(self, paciencia=15, min_cambio=0.005):

        self.paciencia = paciencia
        self.min_cambio = min_cambio
        self.best_metric = None
        self.contador = 0
        self.early_stop = False

    def __call__(self, dato, model, path='guardado.pth'):
        if self.best_metric is None:
            self.best_metric = dato
            torch.save(model.state_dict(), path)
        elif (dato - self.best_metric) > self.min_cambio:
            self.best_metric = dato
            torch.save(model.state_dict(), path)
            self.contador = 0
        else:
            self.contador += 1
            print(f"Contador earlystopping: {self.contador} sobre {self.paciencia}")
            if self.contador >= self.paciencia:
                self.early_stop = True

In [15]:
perdidas_train = []
perdidas_val = []
dice_train = []
dice_val = []
num_epochs = 200
stop = EarlyStopping(paciencia=15, min_cambio=0.005)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    running_dice = 0.0
    
    for images, masks in dataloader_train:
        images = images.to(device)
        masks = masks.to(device)

        outputs = model(images)
        loss = criterion(outputs, masks)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        predictions = torch.sigmoid(outputs)
        masks = masks.long()
        
        dice = dice_metric(predictions, masks)
        
        running_dice += dice.item()

    perdida = running_loss/len(dataloader_train)
    perdidas_train.append(perdida)

    fiabilidad = running_dice/len(dataloader_train)
    dice_train.append(fiabilidad)
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {perdida}, Dice: {fiabilidad}')

    model.eval()
    val_loss = 0.0
    val_dice = 0.0
    
    with torch.no_grad():
        for images, masks in dataloader_val:
            images = images.to(device)
            masks = masks.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, masks)
            
            val_loss += loss.item()

            predictions = torch.sigmoid(outputs)
            predictions = (predictions > 0.5).float()
            masks = masks.long()
          
            batch_dice = 0.0
            for i in range(images.size(0)):
                if masks[i].sum() == 0 and predictions[i].sum() == 0:
                    dice = torch.tensor(1.0)
                else:
                    dice = dice_metric(predictions[i], masks[i])
    
                batch_dice += dice.item()
            
            batch_dice_media = batch_dice / images.size(0)
            val_dice += batch_dice_media
    
    perdida = val_loss/len(dataloader_val)
    perdidas_val.append(perdida)

    fiabilidad = val_dice/len(dataloader_val)
    dice_val.append(fiabilidad)
    
    print(f'Validation Loss: {perdida}, Validation Dice: {fiabilidad}')

    stop(fiabilidad, model, path='UNet_mejor_Axial.pth')
    if stop.early_stop:
        print("Early stopping")
        break

Epoch [1/200], Loss: 0.3258413092022942, Dice: 0.0035142012748110574
Validation Loss: 0.2074333048529095, Validation Dice: 0.4036679179989733
Epoch [2/200], Loss: 0.16511549656429597, Dice: 6.835394519692918e-05
Validation Loss: 0.14255448347992367, Validation Dice: 0.3759357923350358
Contador earlystopping: 1 sobre 15
Epoch [3/200], Loss: 0.11273780020494614, Dice: 0.00011875598506923134
Validation Loss: 0.09459889473186599, Validation Dice: 0.39807673942771177
Contador earlystopping: 2 sobre 15
Epoch [4/200], Loss: 0.08143493040434775, Dice: 0.1921507490828404
Validation Loss: 0.07072157826688555, Validation Dice: 0.5094103658365957
Epoch [5/200], Loss: 0.060511571385206714, Dice: 0.5563200289203275
Validation Loss: 0.05140886745519108, Validation Dice: 0.600471228906557
Epoch [6/200], Loss: 0.045970489751667745, Dice: 0.6709353808433779
Validation Loss: 0.04828981144560708, Validation Dice: 0.6183236441053547
Epoch [7/200], Loss: 0.03614054884641401, Dice: 0.7216739135403787
Validat

Coronal

In [16]:
carpeta = 'Resultados2/Cerebro/Coronal/'
lista_ficheros_cerebro_coronal = obtener_ficheros(carpeta)
print(len(lista_ficheros_cerebro_coronal))

9504


In [17]:
carpeta = 'Resultados2/Mascara/Coronal/'
lista_ficheros_mascara_coronal = obtener_ficheros(carpeta)
print(len(lista_ficheros_mascara_coronal))

9504


In [18]:
carpeta = 'Resultados2/Cerebro/Coronal/Val/'
lista_ficheros_cerebro_coronal_val = obtener_ficheros(carpeta)
print(len(lista_ficheros_cerebro_coronal_val))

1320


In [19]:
carpeta = 'Resultados2/Mascara/Coronal/Val/'
lista_ficheros_mascara_coronal_val = obtener_ficheros(carpeta)
print(len(lista_ficheros_mascara_coronal_val))

1320


In [20]:
train_dataset = Custom_Dataset(lista_ficheros_cerebro_coronal,lista_ficheros_mascara_coronal, argument=True)
dataloader_train = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [21]:
val_dataset = Custom_Dataset(lista_ficheros_cerebro_coronal_val,lista_ficheros_mascara_coronal_val)
dataloader_val = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [22]:
model = smp.Unet(
    encoder_name="resnet34",
    encoder_weights="imagenet",
    in_channels=1,
    classes=1
)

In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [24]:
model = model.to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
dice_metric = Dice(average='micro').to(device)

In [25]:
perdidas_train = []
perdidas_val = []
dice_train = []
dice_val = []
num_epochs = 200
stop = EarlyStopping(paciencia=15, min_cambio=0.005)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    running_dice = 0.0
    
    for images, masks in dataloader_train:
        images = images.to(device)
        masks = masks.to(device)

        outputs = model(images)
        loss = criterion(outputs, masks)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        predictions = torch.sigmoid(outputs)
        masks = masks.long()
        
        dice = dice_metric(predictions, masks)
        
        running_dice += dice.item()

    perdida = running_loss/len(dataloader_train)
    perdidas_train.append(perdida)

    fiabilidad = running_dice/len(dataloader_train)
    dice_train.append(fiabilidad)
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {perdida}, Dice: {fiabilidad}')

    model.eval()
    val_loss = 0.0
    val_dice = 0.0
    
    with torch.no_grad():
        for images, masks in dataloader_val:
            images = images.to(device)
            masks = masks.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, masks)
            
            val_loss += loss.item()

            predictions = torch.sigmoid(outputs)
            predictions = (predictions > 0.5).float()
            masks = masks.long()
          
            batch_dice = 0.0
            for i in range(images.size(0)):
                if masks[i].sum() == 0 and predictions[i].sum() == 0:
                    dice = torch.tensor(1.0)
                else:
                    dice = dice_metric(predictions[i], masks[i])
    
                batch_dice += dice.item()
            
            batch_dice_media = batch_dice / images.size(0)
            val_dice += batch_dice_media
    
    perdida = val_loss/len(dataloader_val)
    perdidas_val.append(perdida)

    fiabilidad = val_dice/len(dataloader_val)
    dice_val.append(fiabilidad)
    
    print(f'Validation Loss: {perdida}, Validation Dice: {fiabilidad}')

    stop(fiabilidad, model, path='UNet_mejor_Coronal.pth')
    if stop.early_stop:
        print("Early stopping")
        break

Epoch [1/200], Loss: 0.2734535199483769, Dice: 0.012386183217015498
Validation Loss: 0.16145781037353335, Validation Dice: 0.3327960866880381
Epoch [2/200], Loss: 0.1252636353941572, Dice: 0.005467550489768673
Validation Loss: 0.10035944339774903, Validation Dice: 0.3552980866148746
Epoch [3/200], Loss: 0.08205962746135341, Dice: 0.10686259441132011
Validation Loss: 0.06503260667834963, Validation Dice: 0.478528403673443
Epoch [4/200], Loss: 0.05597519444539243, Dice: 0.5007606150719943
Validation Loss: 0.04632593123685746, Validation Dice: 0.5390055131689656
Epoch [5/200], Loss: 0.041081808357430784, Dice: 0.6105168790825262
Validation Loss: 0.03403369567933537, Validation Dice: 0.5993709219772081
Epoch [6/200], Loss: 0.030748379047985046, Dice: 0.6931002488072286
Validation Loss: 0.025902260183578447, Validation Dice: 0.6226832552208743
Epoch [7/200], Loss: 0.024045969215815498, Dice: 0.7265975327299745
Validation Loss: 0.020459043305544627, Validation Dice: 0.6257431557327177
Contad

Sagital

In [17]:
carpeta = 'Resultados2/Cerebro/Sagital/'
lista_ficheros_cerebro_sagital = obtener_ficheros(carpeta)
print(len(lista_ficheros_cerebro_sagital))

7920


In [18]:
carpeta = 'Resultados2/Mascara/Sagital/'
lista_ficheros_mascara_sagital = obtener_ficheros(carpeta)
print(len(lista_ficheros_mascara_sagital))

7920


In [19]:
carpeta = 'Resultados2/Cerebro/Sagital/Val/'
lista_ficheros_cerebro_sagital_val = obtener_ficheros(carpeta)
print(len(lista_ficheros_cerebro_sagital_val))

1100


In [20]:
carpeta = 'Resultados2/Mascara/Sagital/Val/'
lista_ficheros_mascara_sagital_val = obtener_ficheros(carpeta)
print(len(lista_ficheros_mascara_sagital_val))

1100


In [21]:
train_dataset = Custom_Dataset(lista_ficheros_cerebro_sagital,lista_ficheros_mascara_sagital, argument=True)
dataloader_train = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [22]:
val_dataset = Custom_Dataset(lista_ficheros_cerebro_sagital_val,lista_ficheros_mascara_sagital_val)
dataloader_val = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [23]:
model = smp.Unet(
    encoder_name="resnet34",
    encoder_weights="imagenet",
    in_channels=1,
    classes=1
)

In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [25]:
model = model.to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
dice_metric = Dice(average='micro').to(device)

In [26]:
perdidas_train = []
perdidas_val = []
dice_train = []
dice_val = []
num_epochs = 200
stop = EarlyStopping(paciencia=15, min_cambio=0.005)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    running_dice = 0.0
    
    for images, masks in dataloader_train:
        images = images.to(device)
        masks = masks.to(device)

        outputs = model(images)
        loss = criterion(outputs, masks)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        predictions = torch.sigmoid(outputs)
        masks = masks.long()
        
        dice = dice_metric(predictions, masks)
        
        running_dice += dice.item()

    perdida = running_loss/len(dataloader_train)
    perdidas_train.append(perdida)

    fiabilidad = running_dice/len(dataloader_train)
    dice_train.append(fiabilidad)
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {perdida}, Dice: {fiabilidad}')

    model.eval()
    val_loss = 0.0
    val_dice = 0.0
    
    with torch.no_grad():
        for images, masks in dataloader_val:
            images = images.to(device)
            masks = masks.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, masks)
            
            val_loss += loss.item()

            predictions = torch.sigmoid(outputs)
            predictions = (predictions > 0.5).float()
            masks = masks.long()
          
            batch_dice = 0.0
            for i in range(images.size(0)):
                if masks[i].sum() == 0 and predictions[i].sum() == 0:
                    dice = torch.tensor(1.0)
                else:
                    dice = dice_metric(predictions[i], masks[i])
    
                batch_dice += dice.item()
            
            batch_dice_media = batch_dice / images.size(0)
            val_dice += batch_dice_media
    
    perdida = val_loss/len(dataloader_val)
    perdidas_val.append(perdida)

    fiabilidad = val_dice/len(dataloader_val)
    dice_val.append(fiabilidad)
    
    print(f'Validation Loss: {perdida}, Validation Dice: {fiabilidad}')

    stop(fiabilidad, model, path='UNet_mejor_Sagital.pth')
    if stop.early_stop:
        print("Early stopping")
        break

Epoch [1/200], Loss: 0.31915115276651995, Dice: 0.0009387563154192439
Validation Loss: 0.23185249666372934, Validation Dice: 0.1362847222222222
Epoch [2/200], Loss: 0.1608274283428346, Dice: 0.09527090181290987
Validation Loss: 0.12415655205647151, Validation Dice: 0.4512565644511563
Epoch [3/200], Loss: 0.10082012769435683, Dice: 0.4826755455184367
Validation Loss: 0.08250750808252229, Validation Dice: 0.5461131843920966
Epoch [4/200], Loss: 0.06898481957614422, Dice: 0.5763644898130048
Validation Loss: 0.05715317506757048, Validation Dice: 0.5682726229254816
Epoch [5/200], Loss: 0.04950353025548881, Dice: 0.6424192569909557
Validation Loss: 0.04141041822731495, Validation Dice: 0.586267039018114
Epoch [6/200], Loss: 0.03729880129497859, Dice: 0.6840689624509504
Validation Loss: 0.03177618928667572, Validation Dice: 0.6061819069549933
Epoch [7/200], Loss: 0.029313461239179297, Dice: 0.7119298846490921
Validation Loss: 0.02513159542447991, Validation Dice: 0.618950988263047
Epoch [8/20

Testeo axial

In [9]:
def crear_lista_pacientes_test(carpeta):
    pruebas = []
    for foto in os.listdir(carpeta):
        partes = foto.split('_')
        paciente = '_'.join(partes[:2])
        if paciente not in pruebas:
            pruebas.append(paciente)
    return natsorted(pruebas)

In [10]:
def crear_diccionario_pruebas_test(pruebas):
    return {clave: [] for clave in pruebas}

In [11]:
model = smp.Unet(
    encoder_name="resnet34",
    encoder_weights="imagenet",
    in_channels=1,
    classes=1
)

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [13]:
model = model.to(device)
dice_metric = Dice(average='micro').to(device)

In [14]:
model.load_state_dict(torch.load("UNet_mejor_Axial.pth"))

/tmp/ipykernel_3932771/2225227020.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("UNet_mejor_Axial.pth"))


<All keys matched successfully>

In [15]:
carpeta = 'Resultados2/Cerebro/Axial/Train/'
lista_ficheros_cerebro_axial_test = obtener_ficheros(carpeta)
print(len(lista_ficheros_cerebro_axial_test))

7920


In [16]:
carpeta = 'Resultados2/Mascara/Axial/Train/'
lista_ficheros_mascara_axial_test = obtener_ficheros(carpeta)
print(len(lista_ficheros_mascara_axial_test))

7920


In [17]:
test_dataset = Custom_Dataset(lista_ficheros_cerebro_axial_test, lista_ficheros_mascara_axial_test)
dataloader_test = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [18]:
carpeta = 'Resultados2/Cerebro/Axial/Train/'
pruebas = crear_lista_pacientes_test(carpeta)
diccionario = crear_diccionario_pruebas_test(pruebas)

dicc_dice = crear_diccionario_pruebas_test(pruebas)

In [19]:
model = model.to(device)
model.eval()

for idx, (images, masks) in enumerate(dataloader_test):
    images = images.to(device)
    masks = masks.to(device)
    
    with torch.no_grad():
        outputs = model(images)
        predictions = torch.sigmoid(outputs)
        predictions = (predictions > 0.5).float()
        masks = masks.long()
        
        if masks.sum() == 0 and predictions.sum() == 0:
            dice = torch.tensor(1.0)
        else:
            dice = dice_metric(predictions, masks)

        dice = dice.item()
    
    original = images.squeeze().cpu().numpy()
    mascara = masks.squeeze().cpu().numpy()
    prediccion = predictions.squeeze().cpu().numpy()

    filename = lista_ficheros_cerebro_axial_test[idx]
    partes = os.path.basename(filename).split('_')
    paciente = '_'.join(partes[:2])

    if paciente in diccionario:
        diccionario[paciente].append(prediccion)
        dicc_dice[paciente].append(dice)
    
for paciente, dice_scores in dicc_dice.items():
    dicc_dice[paciente] = np.mean(dice_scores)

for paciente, dice_score in dicc_dice.items():
    print(f'Paciente: {paciente}, Dice: {dice_score}')

Paciente: P01_T1, Dice: 0.856631065769629
Paciente: P01_T2, Dice: 0.866338690573519
Paciente: P01_T3, Dice: 0.8718853359872645
Paciente: P02_T1, Dice: 0.9162795172496275
Paciente: P02_T4, Dice: 0.8654550704089078
Paciente: P03_T2, Dice: 0.7278364877131852
Paciente: P03_T3, Dice: 0.7197776538404551
Paciente: P03_T4, Dice: 0.7831628516316413
Paciente: P04_T1, Dice: 0.863121809200807
Paciente: P04_T2, Dice: 0.8168698324398561
Paciente: P04_T3, Dice: 0.8338053532622077
Paciente: P05_T1, Dice: 0.7270231313326142
Paciente: P05_T2, Dice: 0.7810172267935492
Paciente: P06_T1, Dice: 0.7452213399112224
Paciente: P06_T2, Dice: 0.7777078674598173
Paciente: P06_T3, Dice: 0.7998392900621349
Paciente: P07_T2, Dice: 0.8606840328736739
Paciente: P08_T2, Dice: 0.7661454138430682
Paciente: P09_T2, Dice: 0.7869380380619656
Paciente: P09_T3, Dice: 0.8059726994146
Paciente: P10_T1, Dice: 0.732842153581706
Paciente: P10_T2, Dice: 0.7476686996492472
Paciente: P11_T2, Dice: 0.8412520600990816
Paciente: P12_T1, 

In [20]:
carpeta = 'Datos/Mascara/'
for paciente, imagenes in diccionario.items():
  
    prueba = [f for f in os.listdir(carpeta) if paciente in f][0]
    ruta = os.path.join(carpeta, prueba)
    imagen_nifti = nib.load(ruta)
    datos = imagen_nifti.get_fdata()
    
    num_index = datos.shape[2]
    num_añadir = porcentaje(num_index, 20)

    original_height, original_width = datos.shape[:2]

    imagenes_redimensionadas = np.zeros((num_index, original_height, original_width))
    
    for idx, img in enumerate(imagenes):
        img_resized = resize(img, (original_height, original_width), mode='constant', preserve_range=True)
        img_resized = (img_resized > 0.5).astype(np.float32)
        imagenes_redimensionadas[idx + num_añadir] = img_resized

    pred_masks_np = np.stack(imagenes_redimensionadas, axis=-1)
    print(pred_masks_np.shape)
    
    nifti_img = nib.Nifti1Image(pred_masks_np, np.eye(4))

    carpeta_nifti = 'Prediccion/Unet/AxialT'
    if not os.path.exists(carpeta_nifti):
        os.makedirs(carpeta_nifti)
    
    nib.save(nifti_img, os.path.join(carpeta_nifti, f'{paciente}_prediccion_Axial.nii.gz'))

(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 21

Testeo coronal

In [21]:
model = smp.Unet(
    encoder_name="resnet34",
    encoder_weights="imagenet",
    in_channels=1,
    classes=1
)

In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [23]:
model = model.to(device)
dice_metric = Dice(average='micro').to(device)

In [24]:
model.load_state_dict(torch.load("UNet_mejor_Coronal.pth"))

/tmp/ipykernel_3932771/2884888535.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("UNet_mejor_Coronal.pth"))


<All keys matched successfully>

In [25]:
carpeta = 'Resultados2/Cerebro/Coronal/Train/'
lista_ficheros_cerebro_coronal_test = obtener_ficheros(carpeta)
print(len(lista_ficheros_cerebro_coronal_test))

9504


In [26]:
carpeta = 'Resultados2/Mascara/Coronal/Train/'
lista_ficheros_mascara_coronal_test = obtener_ficheros(carpeta)
print(len(lista_ficheros_mascara_coronal_test))

9504


In [27]:
test_dataset = Custom_Dataset(lista_ficheros_cerebro_coronal_test, lista_ficheros_mascara_coronal_test)
dataloader_test = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [28]:
carpeta = 'Resultados2/Cerebro/Coronal/Train/'
pruebas = crear_lista_pacientes_test(carpeta)
diccionario = crear_diccionario_pruebas_test(pruebas)

dicc_dice = crear_diccionario_pruebas_test(pruebas)

In [29]:
model = model.to(device)
model.eval()

for idx, (images, masks) in enumerate(dataloader_test):
    images = images.to(device)
    masks = masks.to(device)
    
    with torch.no_grad():
        outputs = model(images)
        predictions = torch.sigmoid(outputs)
        predictions = (predictions > 0.5).float()
        masks = masks.long()
        
        if masks.sum() == 0 and predictions.sum() == 0:
            dice = torch.tensor(1.0)
        else:
            dice = dice_metric(predictions, masks)

        dice = dice.item()
    
    original = images.squeeze().cpu().numpy()
    mascara = masks.squeeze().cpu().numpy()
    prediccion = predictions.squeeze().cpu().numpy()

    filename = lista_ficheros_cerebro_coronal_test[idx]
    partes = os.path.basename(filename).split('_')
    paciente = '_'.join(partes[:2])

    if paciente in diccionario:
        diccionario[paciente].append(prediccion)
        dicc_dice[paciente].append(dice)
    
for paciente, dice_scores in dicc_dice.items():
    dicc_dice[paciente] = np.mean(dice_scores)

for paciente, dice_score in dicc_dice.items():
    print(f'Paciente: {paciente}, Dice: {dice_score}')

Paciente: P01_T1, Dice: 0.821331993987163
Paciente: P01_T2, Dice: 0.8232855851000006
Paciente: P01_T3, Dice: 0.8647434052192804
Paciente: P02_T1, Dice: 0.7621924871522369
Paciente: P02_T4, Dice: 0.7523285757298722
Paciente: P03_T2, Dice: 0.6549594211815433
Paciente: P03_T3, Dice: 0.6089667686345902
Paciente: P03_T4, Dice: 0.5666636255372203
Paciente: P04_T1, Dice: 0.8529236891278715
Paciente: P04_T2, Dice: 0.8170563025456486
Paciente: P04_T3, Dice: 0.8746486302803863
Paciente: P05_T1, Dice: 0.6911675654577486
Paciente: P05_T2, Dice: 0.659800063592918
Paciente: P06_T1, Dice: 0.7019093330159332
Paciente: P06_T2, Dice: 0.7688216603163517
Paciente: P06_T3, Dice: 0.7981765853861967
Paciente: P07_T2, Dice: 0.8744233079028852
Paciente: P08_T2, Dice: 0.7010535639569615
Paciente: P09_T2, Dice: 0.7703872797728488
Paciente: P09_T3, Dice: 0.7205098011164051
Paciente: P10_T1, Dice: 0.6978256719356234
Paciente: P10_T2, Dice: 0.5660324497430613
Paciente: P11_T2, Dice: 0.8298245593905449
Paciente: P12

In [30]:
carpeta = 'Datos/Mascara/'
for paciente, imagenes in diccionario.items():

    prueba = [f for f in os.listdir(carpeta) if paciente in f][0]
    ruta = os.path.join(carpeta, prueba)
    imagen_nifti = nib.load(ruta)
    datos = imagen_nifti.get_fdata()

    num_index = datos.shape[1]
    num_añadir = porcentaje(num_index, 20)

    original_depth, original_width = datos.shape[0], datos.shape[2]

    imagenes_redimensionadas = np.zeros((num_index, original_depth, original_width))

    for idx, img in enumerate(imagenes):
        img_resized = resize(img, (original_depth, original_width), mode='constant', preserve_range=True)
        img_resized = (img_resized > 0.5).astype(np.float32)
        imagenes_redimensionadas[idx + num_añadir] = img_resized



    pred_masks_np = np.stack(imagenes_redimensionadas, axis=1)
    print(pred_masks_np.shape)

    nifti_img = nib.Nifti1Image(pred_masks_np, np.eye(4))

    carpeta_nifti = 'Prediccion/Unet/CoronalT'
    if not os.path.exists(carpeta_nifti):
        os.makedirs(carpeta_nifti)

    nib.save(nifti_img, os.path.join(carpeta_nifti, f'{paciente}_prediccion_Coronal.nii.gz'))

(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 21

Testeo sagital

In [31]:
model = smp.Unet(
    encoder_name="resnet34",
    encoder_weights="imagenet",
    in_channels=1,
    classes=1
)

In [32]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [33]:
model = model.to(device)
dice_metric = Dice(average='micro').to(device)

In [34]:
model.load_state_dict(torch.load("UNet_mejor_Sagital.pth"))

/tmp/ipykernel_3932771/3274674503.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("UNet_mejor_Sagital.pth"))


<All keys matched successfully>

In [35]:
carpeta = 'Resultados2/Cerebro/Sagital/Train/'
lista_ficheros_cerebro_sagital_test = obtener_ficheros(carpeta)
print(len(lista_ficheros_cerebro_sagital_test))

7920


In [36]:
carpeta = 'Resultados2/Mascara/Sagital/Train/'
lista_ficheros_mascara_sagital_test = obtener_ficheros(carpeta)
print(len(lista_ficheros_mascara_sagital_test))

7920


In [37]:
test_dataset = Custom_Dataset(lista_ficheros_cerebro_sagital_test, lista_ficheros_mascara_sagital_test)
dataloader_test = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [38]:
carpeta = 'Resultados2/Cerebro/Sagital/Train/'
pruebas = crear_lista_pacientes_test(carpeta)
diccionario = crear_diccionario_pruebas_test(pruebas)

dicc_dice = crear_diccionario_pruebas_test(pruebas)

In [39]:
model = model.to(device)
model.eval()

for idx, (images, masks) in enumerate(dataloader_test):
    images = images.to(device)
    masks = masks.to(device)
    
    with torch.no_grad():
        outputs = model(images)
        predictions = torch.sigmoid(outputs)
        predictions = (predictions > 0.5).float()
        masks = masks.long()
        
        if masks.sum() == 0 and predictions.sum() == 0:
            dice = torch.tensor(1.0)
        else:
            dice = dice_metric(predictions, masks)

        dice = dice.item()
    
    original = images.squeeze().cpu().numpy()
    mascara = masks.squeeze().cpu().numpy()
    prediccion = predictions.squeeze().cpu().numpy()

    filename = lista_ficheros_cerebro_sagital_test[idx]
    partes = os.path.basename(filename).split('_')
    paciente = '_'.join(partes[:2])

    if paciente in diccionario:
        diccionario[paciente].append(prediccion)
        dicc_dice[paciente].append(dice)
    
for paciente, dice_scores in dicc_dice.items():
    dicc_dice[paciente] = np.mean(dice_scores)

for paciente, dice_score in dicc_dice.items():
    print(f'Paciente: {paciente}, Dice: {dice_score}')

Paciente: P01_T1, Dice: 0.881278816407377
Paciente: P01_T2, Dice: 0.8580874735658819
Paciente: P01_T3, Dice: 0.8638345098292286
Paciente: P02_T1, Dice: 0.8694498991424388
Paciente: P02_T4, Dice: 0.792356180738319
Paciente: P03_T2, Dice: 0.7363888030702418
Paciente: P03_T3, Dice: 0.7615154873241078
Paciente: P03_T4, Dice: 0.5713857249441472
Paciente: P04_T1, Dice: 0.844052083383907
Paciente: P04_T2, Dice: 0.8417423611676151
Paciente: P04_T3, Dice: 0.8556866439906033
Paciente: P05_T1, Dice: 0.7962352292104201
Paciente: P05_T2, Dice: 0.7609320997514508
Paciente: P06_T1, Dice: 0.7169432069767605
Paciente: P06_T2, Dice: 0.8147803517905149
Paciente: P06_T3, Dice: 0.7626888832246715
Paciente: P07_T2, Dice: 0.8357094379988583
Paciente: P08_T2, Dice: 0.8246772603555159
Paciente: P09_T2, Dice: 0.7817615586248311
Paciente: P09_T3, Dice: 0.7758159996433691
Paciente: P10_T1, Dice: 0.7174764703620564
Paciente: P10_T2, Dice: 0.7292930510911074
Paciente: P11_T2, Dice: 0.8370955277572979
Paciente: P12_

In [40]:
carpeta = 'Datos/Mascara/'
for paciente, imagenes in diccionario.items():

    prueba = [f for f in os.listdir(carpeta) if paciente in f][0]
    ruta = os.path.join(carpeta, prueba)
    imagen_nifti = nib.load(ruta)
    datos = imagen_nifti.get_fdata()

    num_index = datos.shape[0]
    num_añadir = porcentaje(num_index, 20)

    original_height, original_depth = datos.shape[1], datos.shape[2]

    imagenes_redimensionadas = np.zeros((num_index, original_height, original_depth))


    for idx, img in enumerate(imagenes):
        img_resized = resize(img, (original_height, original_depth), mode='constant', preserve_range=True)
        img_resized = (img_resized > 0.5).astype(np.float32)
        imagenes_redimensionadas[idx + num_añadir] = img_resized


    pred_masks_np = np.stack(imagenes_redimensionadas, axis=0)
    print(pred_masks_np.shape)

    nifti_img = nib.Nifti1Image(pred_masks_np, np.eye(4))

    carpeta_nifti = 'Prediccion/Unet/SagitalT'
    if not os.path.exists(carpeta_nifti):
        os.makedirs(carpeta_nifti)

    nib.save(nifti_img, os.path.join(carpeta_nifti, f'{paciente}_prediccion_Sagital.nii.gz'))

(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 21

In [2]:
def copiar_archivos_gz(origen, destino):
    for fichero in os.listdir(origen):
        if fichero.endswith('.gz'):
            ruta_archivo_origen = os.path.join(origen, fichero)
            ruta_archivo_destino = os.path.join(destino, fichero)
            shutil.copy2(ruta_archivo_origen, ruta_archivo_destino)

In [3]:
origen = "Prediccion/Unet/Axial/"
destino = "DICE_EVAL"
copiar_archivos_gz(origen, destino)

In [41]:
def umbralizacion(input_dir1, input_dir2, input_dir3, output_dir):

    os.makedirs(output_dir, exist_ok=True)

    input_dirs = [input_dir1, input_dir2, input_dir3]
    nifti_files = []
    for dir in input_dirs:
        nifti_files.extend([f for f in os.listdir(dir) if f.endswith(".nii") or f.endswith(".nii.gz")])

    unicas = set("_".join(f.split("_")[:2]) for f in nifti_files)
    
    for cadena in unicas:
        matching_files = [f for f in nifti_files if cadena in f]
        voxel_data_list = []

        for file in matching_files:
            for dir in input_dirs:
                if file in os.listdir(dir):
                    filepath = os.path.join(dir, file)
                    nifti_img = nib.load(filepath)
                    voxel_data = nifti_img.get_fdata()
                    voxel_data_list.append(voxel_data)
                    break

        suma_voxeles = np.sum(voxel_data_list, axis=0)

        umbralizado = (suma_voxeles > 1).astype(np.float32)

        umbralizado_nifti = nib.Nifti1Image(umbralizado, affine=nifti_img.affine, header=nifti_img.header)
        umbralizado_filepath = os.path.join(output_dir, f"{cadena}_prediccion_combinado.nii.gz")
        nib.save(umbralizado_nifti, umbralizado_filepath)

In [43]:
input_dir1 = "Prediccion/Unet/AxialT/"
input_dir2 = "Prediccion/Unet/CoronalT/"
input_dir3 = "Prediccion/Unet/SagitalT/"
output_dir = "Prediccion/Unet/umbralizacion"

umbralizacion(input_dir1, input_dir2, input_dir3, output_dir)

In [7]:
origen = "Prediccion/Unet/nifti_umbralizacion/"
destino = "DICE_EVAL"
copiar_archivos_gz(origen, destino)

In [44]:
def votacion(input_dir1, input_dir2, input_dir3, output_dir):

    os.makedirs(output_dir, exist_ok=True)

    input_dirs = [input_dir1, input_dir2, input_dir3]
    nifti_files = []
    for dir in input_dirs:
        nifti_files.extend([f for f in os.listdir(dir) if f.endswith(".nii") or f.endswith(".nii.gz")])

    unicas = set("_".join(f.split("_")[:2]) for f in nifti_files)
    
    for cadena in unicas:
        matching_files = [f for f in nifti_files if cadena in f]
        voxel_data_list = []

        for file in matching_files:
            for dir in input_dirs:
                if file in os.listdir(dir):
                    filepath = os.path.join(dir, file)
                    nifti_img = nib.load(filepath)
                    voxel_data = nifti_img.get_fdata()
                    voxel_data_list.append((voxel_data > 0).astype(int))
                    break

        suma_voxeles = np.sum(voxel_data_list, axis=0)

        umbralizado = (suma_voxeles >= 2).astype(np.float32)

        umbralizado_nifti = nib.Nifti1Image(umbralizado, affine=nifti_img.affine, header=nifti_img.header)
        umbralizado_filepath = os.path.join(output_dir, f"{cadena}_prediccion_combinado.nii.gz")
        nib.save(umbralizado_nifti, umbralizado_filepath)

In [45]:
input_dir1 = "Prediccion/Unet/AxialT/"
input_dir2 = "Prediccion/Unet/CoronalT/"
input_dir3 = "Prediccion/Unet/SagitalT/"
output_dir = "Prediccion/Unet/votacion"

votacion(input_dir1, input_dir2, input_dir3, output_dir)

In [46]:
def votacion_ponderada(input_dir1, input_dir2, input_dir3, output_dir, dice_scores):

    os.makedirs(output_dir, exist_ok=True)
    
    input_dirs = [input_dir1, input_dir2, input_dir3]
    nifti_files = []
    for dir in input_dirs:
        nifti_files.extend([f for f in os.listdir(dir) if f.endswith(".nii") or f.endswith(".nii.gz")])
    
    unicas = set("_".join(f.split("_")[:2]) for f in nifti_files)
    
    factor_escala = 3 / sum(dice_scores)
    pesos = [d * factor_escala for d in dice_scores]
    print(pesos)
    for cadena in unicas:
        matching_files = {dir: [f for f in os.listdir(dir) if cadena in f] for dir in input_dirs}
        
        voxel_data_list = []
        
        for i, dir in enumerate(input_dirs):
            for file in matching_files[dir]:
                filepath = os.path.join(dir, file)
                nifti_img = nib.load(filepath)
                voxel_data = nifti_img.get_fdata()
                voxel_data_list.append((voxel_data > 0).astype(float) * pesos[i])
        
        suma_ponderada = np.sum(voxel_data_list, axis=0)
        
        umbralizado = (suma_ponderada >= 1.5).astype(np.float32)
        
        umbralizado_nifti = nib.Nifti1Image(umbralizado, affine=nifti_img.affine, header=nifti_img.header)
        umbralizado_filepath = os.path.join(output_dir, f"{cadena}_prediccion_combinado.nii.gz")
        nib.save(umbralizado_nifti, umbralizado_filepath)

In [47]:
input_dir1 = "Prediccion/Unet/AxialT/"
input_dir2 = "Prediccion/Unet/CoronalT/"
input_dir3 = "Prediccion/Unet/SagitalT/"
output_dir = "Prediccion/Unet/votacion_ponderada"
dice_scores = [0.756752,0.747383,0.751872]

votacion_ponderada(input_dir1, input_dir2, input_dir3, output_dir, dice_scores)

[1.0063160265016908, 0.9938572885633777, 0.9998266849349314]


In [6]:
def umbralizacion_2D_3D(input_dir1, input_dir2, input_dir3, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    
    lista_axial = natsorted([f for f in os.listdir(input_dir1) if f.endswith(".nii.gz")])
    lista_coronal = natsorted([f for f in os.listdir(input_dir2) if f.endswith(".nii.gz")])
    lista_sagital = natsorted([f for f in os.listdir(input_dir3) if f.endswith(".nii.gz")])

    print(len(lista_axial), len(lista_coronal), len(lista_sagital))
    
    for i in range(len(lista_axial)):
        paciente = "_".join(lista_axial[i].split('_')[:2])
        
        ruta_axial = os.path.join(input_dir1, lista_axial[i])
        ruta_coronal = os.path.join(input_dir2, lista_coronal[i])
        ruta_sagital = os.path.join(input_dir3, lista_sagital[i])
        
        nifti_axial = nib.load(ruta_axial)
        nifti_coronal = nib.load(ruta_coronal)
        nifti_sagital = nib.load(ruta_sagital)
        
        voxel_data_list = [nifti_axial.get_fdata(), nifti_coronal.get_fdata(), nifti_sagital.get_fdata()]
        
        suma_voxeles = np.sum(voxel_data_list, axis=0)
        
        umbralizado = (suma_voxeles > 1).astype(np.float32)
        
        umbralizado_nifti = nib.Nifti1Image(umbralizado, affine=nifti_axial.affine, header=nifti_axial.header)
        
        output_filepath = os.path.join(output_dir, f"{paciente}_combinado_PRED.nii.gz")
        nib.save(umbralizado_nifti, output_filepath)

In [7]:
input1 = "Prediccion/Unet/umbralizacion/"
input2 = "Prediccion/Unet++/umbralizacion/"
input3 = "nnUNet/nnUNet_raw/Dataset022_Esclerosis/predictions/"
output = "Prediccion/umbralizacion_2D_3D"
umbralizacion_2D_3D(input1, input2, input3, output)

82 82 82
